<a href="https://colab.research.google.com/github/Bzrm/MidRusNLP/blob/master/embeddings/Embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install conllu

In [1]:
from conllu import parse
import gensim
from tqdm import tqdm

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd /content/drive/My Drive/

/content/drive/My Drive


In [3]:
with open('TRAIN_25.txt', 'r') as f:
  text = f.read()

In [32]:
text[:1000]

'1\t{л. 279}\t_\tX\t_\t_\t0\troot\t_\t_\n2\tСимонова\t_\tADJ\t_\tCase=Nom|Degree=Pos|Gender=Fem|Number=Sing|Variant=Short\t1\tamod\t_\t_\n3\t.\t_\tPUNCT\t_\t_\t1\tpunct\t_\t_\n\n1\tСе\t_\tPART\t_\t_\t2\tadvmod\t_\t_\n2\tяз\t_\tPRON\t_\tCase=Nom|Number=Sing|Person=1|PronType=Prs\t0\troot\t_\t_\n3\tпречистыа\t_\tADJ\t_\tCase=Gen|Degree=Pos|Gender=Fem|Number=Sing|Variant=Long\t4\tamod\t_\t_\n4\tБогородици\t_\tNOUN\t_\tCase=Gen|Gender=Fem|Number=Sing\t2\tnmod\t_\t_\n5\tКузьмина\t_\tADJ\t_\tCase=Gen|Degree=Pos|Gender=Masc|Number=Sing|Variant=Short\t6\tamod\t_\t_\n6\tмонастыря\t_\tNOUN\t_\tCase=Gen|Gender=Masc|Number=Sing\t4\tnmod\t_\t_\n7\tигумен\t_\tNOUN\t_\tCase=Nom|Gender=Masc|Number=Sing\t4\tappos\t_\t_\n8\tПротасей\t_\tPROPN\t_\tCase=Nom|Gender=Masc|Number=Sing\t7\tappos\t_\t_\n9\tс\t_\tADP\t_\t_\t11\tcase\t_\t_\n10\tсвоею\t_\tDET\t_\tCase=Ins|Gender=Fem|Number=Sing|PronType=Prs|Reflex=Yes\t11\tdet\t_\t_\n11\tбратьею\t_\tNOUN\t_\tCase=Ins|Gender=Fem|Number=Sing\t7\tnmod\t_\t_\n12\t,\t_

In [4]:
sentences = parse(text)

In [6]:
len(sentences)

69133

In [5]:
sents = [[word['form'] for word in sent] for sent in sentences]

W2V модель

In [7]:
model = gensim.models.Word2Vec(sents, size = 300, window = 10, min_count = 2, sg = 0)

In [40]:
model.wv.most_similar(positive = ['се'], topn = 20)

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('сице', 0.8249179720878601),
 ('глаголя', 0.8246446847915649),
 ('сие', 0.8110061287879944),
 ('рече', 0.8061538338661194),
 ('Господь', 0.8005088567733765),
 ('глаголюще', 0.797787070274353),
 ('сия', 0.7971638441085815),
 ('Обратитеся', 0.7946710586547852),
 ('Господи', 0.7923119068145752),
 ('нашь', 0.7909238338470459),
 ('владыко', 0.7904869318008423),
 ('любимая', 0.7886483669281006),
 ('«', 0.7868678569793701),
 ('"', 0.786328911781311),
 ('македонское', 0.7855613231658936),
 ('перский', 0.7840856313705444),
 ('Иисус', 0.7836588025093079),
 ('тя', 0.7821434140205383),
 ('мя', 0.7817447185516357),
 ('угодно', 0.7771409749984741)]

Расстояние Левенштейна

In [9]:
from Levenshtein import distance

In [34]:
!pip install python-Levenshtein

     |████████████████████████████████| 51kB 1.9MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.0-cp36-cp36m-linux_x86_64.whl size=144788 sha256=024e7435f3f4fc34b4cc8ff5a272cb8f8b9f7f6d4621270568538631bccb2a2e
  Stored in directory: /root/.cache/pip/wheels/de/c2/93/660fd5f7559049268ad2dc6d81c4e39e9e36518766eaf7e342
Successfully built python-Levenshtein


In [10]:
distance('князя', 'царя')

4

Ближайшие слова к каждому слову

In [11]:
replacements = {}

In [13]:
for sent in tqdm(sents[:10000]):
  for word in sent:
    if word not in replacements:
      replacements[word] = []
    try:
      close = model.wv.most_similar(positive = [word], topn = 10)
      for token in close:
        if token[1] >= 0.75 and distance(word, token[0]) <= 2:
          #print(word, token, distance(word, token[0]))
          if token[0] not in replacements[word]:
            replacements[word].append(token[0])
    except:
      continue

  0%|          | 0/10000 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
100%|██████████| 10000/10000 [39:26<00:00,  4.22it/s]


In [37]:
len(replacements)

39354

Оставляем в словаре только те слова, к которым нашлись такие ближайшие

In [19]:
clean = {}

In [20]:
for word in replacements.keys():
  if replacements[word] != []:
    clean[word] = replacements[word]

In [22]:
len(clean)

2752

In [38]:
clean

{'О': ['НА'],
 'КАКО': ['КАКЪ'],
 'НА': ['КЪ'],
 'ЕМУ': ['НЕМУ'],
 'ВЕЛИКОГО': ['ВЕЛИКАГО', 'ВЕЛИКОМЪ'],
 'ЕГО': ['ЕВО'],
 'велики': ['великы', 'великии', 'великий'],
 'всеа': ['всея'],
 'Руси': ['Росии', 'Русии'],
 'прииде': ['поиде', 'пріиде'],
 'князю': ['Князю'],
 'мѣсяца': ['Мѣсяца'],
 'въ': ['объ'],
 '12': ['13', '7', '8', '9', '15', '14', '30', '11', '6', '5'],
 'три': ['ржи'],
 'дни': ['дне'],
 'людьми': ['людми'],
 'же': ['бо'],
 'посла': ['послаша', 'присла'],
 'князя': ['кнꙗзѧ', 'кнѧзѧ'],
 'Андрѣя': ['Ондрѣя', 'Андрѣа'],
 'братия': ['братие'],
 'князь': ['Князь'],
 'своих': ['твоих'],
 'тое': ['те', 'тою'],
 'тогда': ['егда'],
 'имъ': ['тѣмъ', 'ихъ'],
 'Федорович': ['Федоровичь'],
 'люди': ['людей'],
 'дворяня': ['дворяне'],
 'великого': ['вѣликого'],
 'боярские': ['боярскые'],
 'приидоша': ['пріидоша', 'поидоша'],
 '15': ['13', '12', '14', '11', '20', '30', '8', '9', '25', '18'],
 'пяток': ['пятокъ'],
 'пожгоша': ['зажгоша'],
 'но': ['бо', 'яко'],
 'града': ['градъ'],
 'взя

In [23]:
import json

In [29]:
with open('Rep_dict.json', 'w', encoding = 'utf-8') as outfile:
    json.dump(clean, outfile, ensure_ascii=False)

In [27]:
a = str(clean)

In [28]:
a

'{\'О\': [\'НА\'], \'КАКО\': [\'КАКЪ\'], \'НА\': [\'КЪ\'], \'ЕМУ\': [\'НЕМУ\'], \'ВЕЛИКОГО\': [\'ВЕЛИКАГО\', \'ВЕЛИКОМЪ\'], \'ЕГО\': [\'ЕВО\'], \'велики\': [\'великы\', \'великии\', \'великий\'], \'всеа\': [\'всея\'], \'Руси\': [\'Росии\', \'Русии\'], \'прииде\': [\'поиде\', \'пріиде\'], \'князю\': [\'Князю\'], \'мѣсяца\': [\'Мѣсяца\'], \'въ\': [\'объ\'], \'12\': [\'13\', \'7\', \'8\', \'9\', \'15\', \'14\', \'30\', \'11\', \'6\', \'5\'], \'три\': [\'ржи\'], \'дни\': [\'дне\'], \'людьми\': [\'людми\'], \'же\': [\'бо\'], \'посла\': [\'послаша\', \'присла\'], \'князя\': [\'кнꙗзѧ\', \'кнѧзѧ\'], \'Андрѣя\': [\'Ондрѣя\', \'Андрѣа\'], \'братия\': [\'братие\'], \'князь\': [\'Князь\'], \'своих\': [\'твоих\'], \'тое\': [\'те\', \'тою\'], \'тогда\': [\'егда\'], \'имъ\': [\'тѣмъ\', \'ихъ\'], \'Федорович\': [\'Федоровичь\'], \'люди\': [\'людей\'], \'дворяня\': [\'дворяне\'], \'великого\': [\'вѣликого\'], \'боярские\': [\'боярскые\'], \'приидоша\': [\'пріидоша\', \'поидоша\'], \'15\': [\'13\', \'12

In [36]:
with open('Rep_dict.txt', 'w', encoding = 'utf-8') as outfile:
    outfile.write(text_f)

In [33]:
text_f = ''

In [34]:
for word in clean.keys():
  line = '{} :  {}'.format(word, ', '.join(clean[word]))
  text_f += line + '\n'

In [35]:
text_f

'О :  НА\nКАКО :  КАКЪ\nНА :  КЪ\nЕМУ :  НЕМУ\nВЕЛИКОГО :  ВЕЛИКАГО, ВЕЛИКОМЪ\nЕГО :  ЕВО\nвелики :  великы, великии, великий\nвсеа :  всея\nРуси :  Росии, Русии\nприиде :  поиде, пріиде\nкнязю :  Князю\nмѣсяца :  Мѣсяца\nвъ :  объ\n12 :  13, 7, 8, 9, 15, 14, 30, 11, 6, 5\nтри :  ржи\nдни :  дне\nлюдьми :  людми\nже :  бо\nпосла :  послаша, присла\nкнязя :  кнꙗзѧ, кнѧзѧ\nАндрѣя :  Ондрѣя, Андрѣа\nбратия :  братие\nкнязь :  Князь\nсвоих :  твоих\nтое :  те, тою\nтогда :  егда\nимъ :  тѣмъ, ихъ\nФедорович :  Федоровичь\nлюди :  людей\nдворяня :  дворяне\nвеликого :  вѣликого\nбоярские :  боярскые\nприидоша :  пріидоша, поидоша\n15 :  13, 12, 14, 11, 20, 30, 8, 9, 25, 18\nпяток :  пятокъ\nпожгоша :  зажгоша\nно :  бо, яко\nграда :  градъ\nвзяша :  взя\nОндрѣй :  Ондрей, Андрей\nнимъ :  нимь\nплен :  пеш\nити :  поити\nсобѣ :  себѣ, себе\nсамъ :  тамо\nбратиею :  братьею, братьєю\nсвоею :  всею\nпоиде :  иде, прииде\nста :  сташа\nсвоемъ :  семъ\nпослаша :  посла, послашася\nреку :  реки, 